In [1]:
import pickle
from datetime import datetime
import pandas as pd 
import os

In [2]:
def load_pickle(data_path):
    with open(data_path,'rb') as f:
        data = pickle.load(f)
    return data
def save_pickle(data,save_path):
    with open(save_path,'wb') as f:
        pickle.dump(data,f)

## 1. convert to Recbole and SFT format

### 1. Recoble format
- category.pickle
- dataset_name.inter
- dataset_name.item

### 2. SFT format
- category.pickle 'category': [item_ids]
- metas.pickle 'item_id': {'item_id', 'title'}
- sequential.pickle 'user_id': [item_ids]

In [21]:
file_path = '/home/wangshuo/codes/rec-scope/constraint_search/data/Toys/base/toys_final.pickle'
final_df = load_pickle(file_path)
final_df.head()

,user_id,interactions
0,1,"[(Hasbro Electronic Catch Phrase, B001RNFQNK, ..."
1,2,"[(Megabloks CAT 3in1 Ride On Truck, B0050K0AQE..."
2,3,"[(Lost Cities, B00005UNAV, [Toys & Games, Game..."
3,4,"[(Mastermind, B00000DMBF, [Toys & Games, Games..."
4,5,"[(Fisher-Price Brilliant Basics Rock-a-Stack, ..."


In [ ]:
final_df.iloc[0]['interactions']

In [11]:
import ast
def covert_to_list(tags_str):
    try:
        tags = ast.literal_eval(tags_str)
        all_tags = []
        for tag in tags:
            all_tags.extend(tag)
        return all_tags
    except:
        return []

In [50]:

sequential = {}
metas ={}
category = {}
iter =[]
item_recbole = []
item_list =[]
COMMON_TAG = 'toys'
miss_count = 0
miss_titile =0 
empty_count = 0

for index, row in final_df.iterrows():
    user_id = str(row['user_id'])
    # interactions(item_title,item_id, timestamp, tags(list))
    interactions = row['interactions']
    item_ids = []

    for interaction in interactions:
        if type(interaction[0]) != str:
            miss_titile += 1
            continue
        item_title = interaction[0].strip()
        item_id = str(interaction[1])
        tags = interaction[2]
        timestamp = interaction[3]
        
        
        if len(tags) == 0:
            tags = [COMMON_TAG]
            miss_count += 1

        for tag in tags:
            tag =tag.strip()
            if tag not in category:
                category[tag] = []
            if item_id not in category[tag]:
                category[tag].append(item_id)
            # category[tag].append(item_id)
        iter.append({"user_id:token":user_id,"item_id:token":item_id,"timestamp:float":float(timestamp)})
        # item_recbole.append({"item_id:token":item_id,"timestamp:float":toTimestamp(timestamp)})
        item_ids.append(item_id)
        metas[item_id]={"title":item_title}
        item_list.append(item_title)
    
    if len(item_ids) <= 3:
        empty_count += 1
        continue
    sequential[user_id]=item_ids
print('miss_count:',miss_count)
print("miss_titile:",miss_titile)
print("empty_count:",empty_count)

miss_count: 17
miss_titile: 787
empty_count: 35


In [26]:
len(metas)

11862

In [52]:
iter_df = pd.DataFrame(iter)
iter_df = iter_df[["user_id:token","item_id:token","timestamp:float"]]
iter_df.to_csv("/home/wangshuo/codes/InstructControllableRec_RLHF/SASRec/dataset/toys/toys.inter",index=False,sep='\t')

In [51]:
save_base_dir = "/home/wangshuo/codes/InstructControllableRec_RLHF/data/dataset/toys/"
save_pickle(sequential,os.path.join(save_base_dir,"sequential.pickle"))
save_pickle(metas,os.path.join(save_base_dir,"metas.pickle"))
save_pickle(category,os.path.join(save_base_dir,"category.pickle"))
save_pickle(item_list,os.path.join(save_base_dir,"items.pickle"))

In [49]:
sequential['10158']

['B007U7M0L8', 'B0015ZITNY', '0911121013']

In [48]:
empty_count=0
for key in sequential.keys():
    if len(sequential[key]) <= 2:
        print(key,sequential[key])
        empty_count += 1
print(empty_count)

0


In [4]:
save_base_dir = "/home/wangshuo/codes/InstructControllableRec_RLHF/data/dataset/steam/"
data = load_pickle(os.path.join(save_base_dir,"metas.pickle"))

In [16]:
len(set(category[COMMON_TAG]))

555

In [ ]:
data1=load_pickle("/home/wangshuo/codes/InstructControllableRec_RLHF/snap/0208-toys-ctrl/Epoch35_SFT.pth_Result_SFTTestSeqRec_Top10.pickle")
data1

In [6]:
data2=load_pickle("/home/wangshuo/codes/rec-scope/constraint_search/data/steam/base/items.pickle")

In [7]:
not_in = []
for item in data1:
    if item not in data2:
        not_in.append(item)
print(len(not_in))

0


In [18]:
'The Witcher® 3: Wild Hunt' in data2

True